In [10]:
import numpy as np
import pandas as pd

### Aufgabe 1
Format für products: Name, Preis, Gewicht

In [11]:
products = np.array(["Edelstein","Epoxidharz","Kupfer"])
profits = np.array([30, 10, 100])
weights = np.array([2, 1, 6])



In [12]:
# Jedes Szenario wird durchlaufen (BruteForce)
def check_constraints(ed, ku, w_total, u_total, w_max, u_max):
    return ku <= 2 * ed and w_total <= w_max and u_total <= u_max

def optimize(u_max, w_max):
    results = []
    u_max = u_max
    for ed in range(u_max):
        for ep in range(u_max):
            for ku in range(u_max):
                u_total = ed + ep + ku
                w_total = weights[0] * ed + weights[1] * ep + weights[2] * ku
                if check_constraints(ed, ku, w_total, u_total, w_max, u_max):
                    profit = profits[0] * ed + profits[1] * ep + profits[2] * ku
                    results.append({
                        "Gewinn": profit,
                        "Gewicht": w_total,
                        "Einheiten": u_total,
                        "Edelstein": ed,
                        "Epoxidharz": ep,
                        "Kupfer": ku
                    })

    return results


In [13]:
max_units = 15
max_weight = 56
optimized_products = optimize(max_units, max_weight)
df = pd.DataFrame(optimized_products)
# Optimized_Products wird nach Gewinn sortiert und es werden nur die "Top 10" angezeigt
sorted_df = df.sort_values(by="Gewinn", ascending=False).reset_index(drop=True).loc[:10]

In [14]:
print(sorted_df)

    Gewinn  Gewicht  Einheiten  Edelstein  Epoxidharz  Kupfer
0      920       56         12          4           0       8
1      910       56         14          7           0       7
2      900       56         15          6           2       7
3      890       55         14          6           1       7
4      880       54         13          6           0       7
5      880       55         15          5           3       7
6      870       54         14          5           2       7
7      870       54         15          9           0       6
8      860       53         13          5           1       7
9      860       54         15          4           4       7
10     850       53         14          4           3       7


## Nicht brute-force mit SciPy
- Wir gehen hier wieder von 0 aus, das heißt die Variablen von der BruteForce methode müssen vorher gecleared werden

In [2]:
from scipy.optimize import linprog
import numpy as np
import pandas as pd

Erklärung zu SciPy:
Linprog steht für Linear Programming, optimize nutzt sämtliche Algorithmen(Die in Methods beschrieben sind, u.a. high-ds: Dualer Simplex) um die Eingabe zu minimieren.
c steht für die Zielfunktion
A_ub steht für die linke Seite der Nebenbedingungen und ub für "<="
A_eq steht für dasselbe nur anstatt "<=" für "="
b_ub steht für die rechte Seite der Nebenbedingungen

In [8]:
products = np.array(["Edelstein", "Epoxidharz", "Kupfer"])
profits = np.array([30, 10, 100])
weights = np.array([2, 1, 6])

u_max = 15
w_max = 56

# Da scipy minimiert, müssen Gewinne negativ gesetzt werden!
c = -profits

# Bedingungen
A_ub = np.array([
    weights,
    np.ones(len(products)), # Summe der Einheiten
    [-2, 0, 1] # Kupfer <= 2 * Edelsteine
])
b_ub = np.array([w_max, u_max, 0])

res = linprog(c,
              A_ub=A_ub,
              b_ub=b_ub,
              method="highs-ds")

if res.success:
    solution = np.floor(res.x)
    df = pd.DataFrame({
        "Gewinn": solution * profits,
        "Produkt": products,
        "Menge": solution.astype(int)
    })
    print(df)
    print(f"Gesamtgewinn: {np.dot(profits, solution)}")
else:
    print("Fehler: ", res.message)


   Gewinn     Produkt  Menge
0   120.0   Edelstein      4
1     0.0  Epoxidharz      0
2   800.0      Kupfer      8
Gesamtgewinn: 920.0


# Aufgabe 2

In [31]:
graph = {
    "A": [("B", 6), ("C", 1), ("D", 1), ("E", 2), ("F", 6)],
    "B": [("D", 3), ("J", 8), ("G", 1)],
    "C": [("E", 4)],
    "D": [("E", 1), ("F", 1), ("H", 3)],
    "E": [("F", 3), ("K", 3), ("L", 5)],
    "F": [("G", 2), ("J", 1), ("K", 1)],
    "G": [("H", 2), ("J", 8)],
    "H": [("I", 5), ("M", 9)],
    "I": [("J", 3), ("K", 4), ("L", 2), ("M", 2)],
    "J": [("K", 2), ("L", 1)],
    "K": [("L", 1), ("N", 7)],
    "L": [("M", 1), ("N", 1)],
    "M": [("N", 3)],
}

def make_undirected(g):
    # list(graph) damit die "Keys" eingefroren werden, wenn graph.items() benutzt wird könnten duplikate entstehen, da der Loop über die neuen Keys iterieren könnte
    for v in list(g):
        for w, cost in g[v]:
            if w not in g:
                # Wenn es w im Graphen noch nicht gibt (Aus der Aufgabe könnte das Beispielsweise N sein, wird der Punkt erstellt
                g[w] = []
            if v not in [n for n, _ in g[w]]:
                # Nur wenn die Verbindung noch nicht existiert, wird sie hinzugefügt um duplikate zu vermeiden
                g[w].append((v, cost))

make_undirected(graph)


def dijkstra(g, start, end):
    # Set um duplikate zu vermeiden
    unvisited = set(g.keys())
    # Distances werden vorerst auf infinite gesetzt, da sie berechnet werden müssen
    dist = { node: float("inf") for node in g }
    prev = {node: None for node in g}

    dist[start] = 0

    while unvisited:
        # Node mit der kleinsten Distanz / Preis finden
        curr = min(
            (node for node in unvisited),
            key=lambda node: dist[node],
            default=None
        )
        if curr is None or dist[curr] == float("inf"):
            # Wenn nichts erreichbar ist wird abgebrochen
            break

        unvisited.remove(curr)

        for neighbor, cost in g[curr]:
            if neighbor in unvisited:
                alt = dist[curr] + cost
                if alt < dist[neighbor]:
                    dist[neighbor] = alt
                    prev[neighbor] = curr


    # Den Path rekonstruieren:
    path = []
    node = end
    while node:
        path.append(node)
        node = prev[node]
    path.reverse()

    return end, path, dist[end]

res_node, res_path, res_dist = dijkstra(graph, "A", "N")
print(f"Wir haben {res_node} erreicht, nachdem wir über {"->".join(res_path)} gegangen sind, die Endkosten betragen: {res_dist}")

Wir haben N erreicht, nachdem wir über A->D->F->K->L->N gegangen sind, die Endkosten betragen: 5
